In [ ]:
#### import necessary libraries :
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
car = pd.read_csv('car.csv')


In [ ]:
car.head()### understaidning the data

In [ ]:
### as the number of attributes and features are less , we could factorize the string attribute from 0 to 1,2,3 etc
car.safety=pd.factorize(car.safety)[0]### low = 0; 1 = mid ; 2=high
car.luggage_boots= pd.factorize(car.luggage_boots)[0]#### small = 0;med = 1 , big=2
car.label = pd.factorize(car.label)[0]### unacc = 0 , acc = 1 , vgood = 2 ,good = 3 

In [ ]:
#### as our factorize will start from 0 so in that case 0 : vhigh , but as it a categorical value so we make an extra list and transform
#### the data from highest to lowest order 
buying = {'vhigh':3,'high':2,'med':1,'low':0}
car.buying =[buying[items]for items in car.buying]

In [ ]:
maintainance = {'vhigh':3,'high':2,'med':1,'low':0}
car.maintainance =[maintainance[items]for items in car.maintainance]

In [ ]:
doors = {'2':2,'3':3,'4':4,'5more':5}
car.doors =[doors[items]for items in car.doors]

In [ ]:
person = {'2':2,'4':4,'more':5}
car.person =[person[items]for items in car.person]

In [ ]:
car.head()

In [ ]:
####normalize the input parameters : 
from sklearn.preprocessing import Normalizer
array = car.values
X = array[:,0:6]####input features , first 6 columns
scaler = Normalizer().fit(X)
X_norm = scaler.transform(X)


In [ ]:
Y =array[:,6]####output 
dum_Y=tf.keras.utils.to_categorical(Y)### we create a categorical dummy variable which detects 
###[1,0,0,0]= unacc , [0,1,0,0]=acc, etc
print (dum_Y)

In [ ]:
###performing a train - test split where 40 % of our data is used to test the data
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_norm,dum_Y,test_size = 0.40 ,random_state=101)

In [ ]:
### building up the model : 
model = tf.keras.models.Sequential()#### we want to have a feed forward network
model.add(tf.keras.layers.Flatten())### flattening the data helps to compress the input in such a which allows the network perform better
### we would want to have 2 hidden layers with 128 neurons and each have rectified linear function as the activation function 
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
### as the labels are 0 to 9 so 10 output nodes are needed and the activation function is softmax function 
model.add(tf.keras.layers.Dense(10,activation = tf.nn.softmax))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(4, activation=tf.nn.softmax))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=200)

In [ ]:
#### evaluation : 
val_loss, val_acc = model.evaluate(X_train, Y_train)
print(val_loss)
print(val_acc)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test,prediction)))